# Import

In [ ]:
import os

os.chdir(os.path.dirname(os.getcwd()))

In [ ]:
from jovis_model.config import Config
from run import ModelRunner

# Bedrock

In [ ]:
# params = {
#     "pkg": "llm",
#     "task": "chat",
#     "use_hf_model": True,
#     "params": {
#         "hf_name": "meta-llama/Meta-Llama-3-8B-Instruct"
#     }
# }
params = {
    "pkg": "llm",
    "task": "bedrock",
    "use_hf_model": False,
    "params": {
    }
}
config = Config(**params)
runner = ModelRunner(
    config=config,
    mode="inference"
)

In [ ]:
dialogs = [
    {"role": "user", "content": "What is you name?"}
]

out = runner.run(
    sample_inputs=dialogs
)